In [1]:
import pymongo
import requests
import json
import time

In [2]:
API_KEY = '42aabe7a7b92bfcf410adf2a04849744'
client = pymongo.MongoClient("mongodb://101.132.171.223:27017/")
db = client['shoprec']
col = db['restaurant']

In [16]:
def getCoord(x):
    time.sleep(1)
    addr = x['address']
    r = requests.get('https://restapi.amap.com/v3/geocode/geo?', params={
        'key': API_KEY,
        'address': addr,
        'city': '武汉'
    })
    r.encoding = 'utf-8'
    content = json.loads(r.text)
    print(content)
    if content['count'] == '0':
        return -1
    lon, lat = content['geocodes'][0]['location'].split(',')
    x['lon'] = lon
    x['lat'] = lat
    return x

In [4]:
res = list(col.find())

In [9]:
data = []
non_data = []

In [17]:
# data = list(map(updateInfo, res))

for i in range(6191, len(res)):
    if res[i]['address'] == '':
        continue
    _data = getCoord(res[i])
    if _data == -1:
        non_data.append(res[i])
        print(i, 'not found')
    else:
        data.append(_data)
        if i % 50 == 0:
            print(i, '/', len(res))

{'info': 'DAILY_QUERY_OVER_LIMIT', 'infocode': '10003', 'status': '0'}


KeyError: 'count'

In [13]:
len(data)

6191

In [20]:
_data = [dict(t) for t in set([tuple(d.items()) for d in data])]

In [21]:
len(_data)

6191

In [23]:
import pandas as pd
df = pd.io.json.json_normalize(_data)

In [24]:
_col = db['rest']
_col.insert_many(_data)